In [3]:
########## HELPER FUNCTIONS ##########
def get_flight_lines():
    """
    Returns sample flight records as a list of strings.
    Format:
    FlightNo Date Time From To Seats(First/Business/Economy) BaseFare Miles
    """
    return [
        "AB123 2025-11-09 15:30 ICN JFK 10/20/50 500 10500.0",
        "CD456 2025-11-10 09:00 ICN LAX 5/10/30 600 12000.0"
    ]

def get_transaction_lines():
    """
    Returns sample transaction records as a list of strings.
    Format:
    TXID Type MBID FlightNo Date Time Payload
    """
    return [
        "T0001 RES MB123 AB123 2025-11-09 15:30 A-05J-10Y-20",
        "T0002 RES MB456 CD456 2025-11-10 09:00 A-02J-05Y-10",
        "T0003 CAN MB123 AB123 2025-11-09 15:30 B0001",
        "T0004 TKT MB456 CD456 2025-11-10 09:00 B0002"
    ]
#######################################

########## COUNT SEATS ##########
def count_seats(classcode):
    first = 0
    business = 0
    economy = 0

    for i in range(0, len(classcode), 4):
        token = classcode[i:i+4]
        if len(token) < 4:
            continue

        booking = token[0]
        count = int(token[2:4])

        if booking in ['A', 'F']:
            first += count
        elif booking in ['J', 'C']:
            business += count
        elif booking in ['Y', 'M', 'K']:
            economy += count

    return (first, business, economy)
#######################################

########## PROCESS TRANSACTION ##########
def process_transaction(transaction, flights, reservations):
    flightid = transaction['no'] + str(transaction['datetime'])
    if flightid not in flights:
        return 'SEARCH ERROR: NO SUCH FLIGHT'

    if transaction['type'] == 'RES':
        flight = flights[flightid]
        f_req, b_req, e_req = count_seats(transaction['payload'])

        if (f_req > flight['seats'][0] or
            b_req > flight['seats'][1] or
            e_req > flight['seats'][2]):
            return 'RESERVATION ERROR: NOT ENOUGH SEATS'

        flight['seats'][0] -= f_req
        flight['seats'][1] -= b_req
        flight['seats'][2] -= e_req

        bid = transaction['txid'].replace('T', 'B', 1)

        reservations[flightid][bid] = {
            'mbid': transaction['mbid'],
            'classcode': transaction['payload'],
            'ticketed': False
        }

        return 'RESERVATION SUCCESS'

    elif transaction['type'] == 'CAN':
        rid = transaction['payload']
        if rid not in reservations[flightid]:
            return 'CANCEL ERROR: NO SUCH RESERVATION'

        res = reservations[flightid][rid]

        if res['mbid'] != transaction['mbid']:
            return 'CANCEL ERROR: NO PERMISSION'

        if res['ticketed']:
            return 'CANCEL ERROR: TICKETED RESERVATION'

        f, b, e = count_seats(res['classcode'])
        flights[flightid]['seats'][0] += f
        flights[flightid]['seats'][1] += b
        flights[flightid]['seats'][2] += e

        del reservations[flightid][rid]

        return 'CANCEL SUCCESS'

    elif transaction['type'] == 'TKT':
        rid = transaction['payload']
        if rid not in reservations[flightid]:
            return 'TICKET ERROR: NO SUCH RESERVATION'

        res = reservations[flightid][rid]

        if res['mbid'] != transaction['mbid']:
            return 'TICKET ERROR: NO PERMISSION'

        res['ticketed'] = True
        return 'TICKET SUCCESS'

    else:
        return 'PARSE ERROR: INVALID TRANSACTION TYPE'
#######################################

########## PARSE FLIGHT RECORD ##########
def parse_flight_record(record, flights):
    parts = record.split()
    no = parts[0]
    date = parts[1].replace('-', '')
    time = parts[2].replace(':', '')
    frm = parts[3]
    to = parts[4]
    seats = list(map(int, parts[5].split('/')))
    basefare = int(parts[6])
    miles = float(parts[7])

    key = f"{no}{date}{time}"
    flights[key] = {
        'no': no,
        'datetime': int(date + time),
        'from': frm,
        'to': to,
        'seats': seats,
        'basefare': basefare,
        'miles': miles
    }
#######################################

########## PARSE TRANSACTION RECORD ##########
def parse_transaction_record(record, transactions):
    parts = record.split()
    txid = parts[0]
    ttype = parts[1]
    mbid = parts[2]
    no = parts[3]
    date = parts[4].replace('-', '')
    time = parts[5].replace(':', '')
    payload = ' '.join(parts[6:])  # join the rest of the string as payload

    transactions.append({
        'txid': txid,
        'type': ttype,
        'mbid': mbid,
        'no': no,
        'datetime': int(date + time),
        'payload': payload
    })

#######################################

########## MAIN FUNCTION ##########
def main():
    flights_lines = get_flight_lines()
    transactions_lines = get_transaction_lines()

    flights = {}
    transactions = []

    for record in flights_lines:
        parse_flight_record(record, flights)
    for record in transactions_lines:
        parse_transaction_record(record, transactions)

    reservations = {key: {} for key in flights.keys()}

    for tx in transactions:
        result = process_transaction(tx, flights, reservations)
        if 'SUCCESS' not in result:
            print(tx['txid'] + ' - ' + result)

    for key, value in reservations.items():
        print(f'Flight {key}: {len(value)} reservations, {flights[key]["seats"]} available')
#######################################

if __name__ == "__main__":
    main()


Flight AB123202511091530: 0 reservations, [10, 20, 50] available
Flight CD456202511100900: 1 reservations, [3, 5, 20] available
